In [ ]:
"""
Singapore Air Traffic Class Prediction Model (TRAINING)

This code builds a machine learning model to predict traffic volume classes for Singapore air travel.

The model helps airlines, travel agencies, and policymakers to:
1. Predict peak and off-peak travel periods
2. Optimize flight schedules
3. Allocate resources efficiently
4. Enhance passenger experience

Author: Myat Ko
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

In [ ]:
# =============================================================================
# DATA LOADING AND EXPLORATION
# =============================================================================
# First, we load our dataset and examine its basic properties

# Attempt to load the dataset from different possible locations
try:
    df = pd.read_csv("train_data_with_traffic_class.csv")
except:
    # Try alternative paths if the file isn't found
    try:
        df = pd.read_csv("C:/Users/Myat Ko/Documents/GitHub/traveltrends/Classification/train_data_with_traffic_class.csv")
    except:
        # To adjust this path based on current file structure
        print("Please check the file path and try again.")

# Print dataset summary information
print(f"Dataset shape: {df.shape}")
print(f"Number of traffic classes: {df['Traffic_Class'].nunique()}")
print(f"Traffic classes: {sorted(df['Traffic_Class'].unique())}")

# Check for missing values
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print("\nMissing values in dataset:")
    print(missing_values[missing_values > 0])

# Examine the distribution of traffic classes
traffic_class_counts = df['Traffic_Class'].value_counts().sort_index()
print("\nTraffic class distribution:")
for class_id, count in traffic_class_counts.items():
    percentage = 100 * count / len(df)
    print(f"Class {class_id}: {count} samples ({percentage:.1f}%)")

In [ ]:
# =============================================================================
# DATA PREPROCESSING AND FEATURE ENGINEERING
# =============================================================================

print("\n=== DATA PREPROCESSING AND FEATURE ENGINEERING ===")

# First, remove any columns with all NaN values
df = df.dropna(axis=1, how='all')

# Only drop Traffic_Class since that's our target
drop_columns = ["Traffic_Class"]

# Identify categorical columns (object type) and numeric columns
categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and col not in drop_columns]
numeric_columns = [col for col in df.columns if df[col].dtype != 'object' and col not in drop_columns]

print(f"Categorical columns: {categorical_columns}")
print(f"Numeric columns: {numeric_columns}")

# =============================================================================
# KEY CHANGE: Use One-Hot Encoding instead of Label Encoding for categorical variables
# =============================================================================

# Handle categorical features with one-hot encoding
# This prevents introducing arbitrary ordinal relationships
if categorical_columns:
    print("\nApplying one-hot encoding to categorical columns...")
    
    # First, fill missing values in categorical columns
    for col in categorical_columns:
        df[col] = df[col].fillna('Unknown')
    
    # Apply one-hot encoding
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_cats = encoder.fit_transform(df[categorical_columns])
    
    # Get the feature names from the encoder
    encoded_feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded values
    encoded_df = pd.DataFrame(encoded_cats, columns=encoded_feature_names, index=df.index)
    
    # Concatenate with the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)
    
    print(f"- Created {len(encoded_feature_names)} one-hot encoded features")
    
    # Track the encoded columns for later use
    categorical_encoded = list(encoded_feature_names)
    # Keep track of original columns to exclude from model features
    categorical_columns_to_drop = categorical_columns.copy()
else:
    categorical_encoded = []
    categorical_columns_to_drop = []

# Define our base feature columns - both numeric and encoded categorical
base_feature_columns = numeric_columns + categorical_encoded

In [ ]:
# =============================================================================
# MODEL PIPELINE SETUP
# =============================================================================
# Create standardized processing pipelines to ensure consistent data handling
# during both training and inference.

print("\n=== MODEL PIPELINE SETUP ===")

# Standard preprocessing pipeline for basic features
# This ensures consistent handling of missing values and feature scaling
preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values with median
    ('scaler', StandardScaler())  # Standardize features to mean=0, std=1
])

# Enhanced preprocessing pipeline that includes polynomial feature generation
# This creates interaction terms to capture complex relationships
poly_preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    # Create interaction terms between features while avoiding full polynomials
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
])

In [ ]:
# =============================================================================
# HYPERPARAMETER TUNING
# =============================================================================
# Find the optimal model parameters through grid search cross-validation
# This systematically tests different combinations to find the best settings.

print("\n=== HYPERPARAMETER TUNING ===")

# Create a complete pipeline including preprocessing and the classifier
pipe = Pipeline([
    ('preprocessor', preprocessing),
    ('classifier', LogisticRegression(max_iter=2000, random_state=42))
])

# Define the parameter grid to search
# These are the key hyperparameters that affect logistic regression performance
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],  # Regularization strength (lower = stronger)
    'classifier__solver': ['lbfgs', 'newton-cg'],  # Optimization algorithm
    'classifier__class_weight': [None, 'balanced']  # Whether to adjust for class imbalance
}

# Use stratified k-fold cross-validation to ensure representative sampling
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create the grid search object
grid_search = GridSearchCV(
    pipe, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1
)

print("Performing grid search for optimal hyperparameters...")
# Fit the grid search to find the best parameters
grid_search.fit(X_train, y_train)

# Display the best parameters and their performance
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

# Store the best parameters for use in our models
best_params = grid_search.best_params_
best_C = best_params['classifier__C']
best_solver = best_params['classifier__solver']
best_class_weight = best_params['classifier__class_weight']

print(f"\nOptimal parameters: C={best_C}, solver={best_solver}, class_weight={best_class_weight}")

In [ ]:
# =============================================================================
# MODEL COMPARISON
# =============================================================================
# Compare different model configurations to understand which approach works best
# This helps identify the impact of feature engineering and hyperparameter tuning.

print("\n=== MODEL COMPARISON ===")

# Define three model variations to compare:
models = {
    # Baseline model with default hyperparameters
    "Base Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(max_iter=2000, random_state=42, solver='lbfgs'))
    ]),
    
    # Model with tuned hyperparameters from grid search
    "Tuned Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ]),
    
    # Model with tuned hyperparameters AND polynomial feature interactions
    "Logistic with Enhanced Features": Pipeline([
        ('preprocessor', poly_preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ])
}

# Function to evaluate each model and produce detailed performance metrics
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Classification Model"):
    """
    Trains and evaluates a classification model for traffic class prediction.
    
    Parameters:
    -----------
    model : The classification model pipeline to evaluate
    X_train, X_test : Training and test feature matrices
    y_train, y_test : Training and test target values
    model_name : Name of the model for reporting purposes
    
    Returns:
    --------
    accuracy : Overall accuracy of the model
    y_pred : Predicted classes for the test set
    trained_model : The trained model object
    """
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate overall accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Print performance summary
    print(f"\n===== {model_name} =====")
    print(f"Accuracy: {accuracy:.4f}")
    
    # Generate detailed classification metrics by class
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Create confusion matrix to visualize prediction patterns
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix as a heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Traffic Class')
    plt.xlabel('Predicted Traffic Class')
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_pred, model

# Storage for results and trained models
results = {}
models_trained = {}

# Loop through each model and evaluate its performance
for name, model in models.items():
    # Evaluate the model using the same X_train and X_test for all models
    accuracy, y_pred, trained_model = evaluate_model(
        model, X_train, X_test, y_train, y_test, name
    )
    results[name] = accuracy
    models_trained[name] = trained_model

# Print comparison of model accuracies, sorted from best to worst
print("\n=== FINAL RESULTS ===")
for name, accuracy in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{name}: {accuracy:.4f}")

In [ ]:
# =============================================================================
# CREATE ADDITIONAL ADVANCED FEATURES
# =============================================================================
print("\n=== CREATING ADVANCED FEATURE COMBINATIONS ===")

# 1. Calculate traffic ratios to capture proportional patterns
if all(col in df.columns for col in ['Arrivals', 'Departures', 'Total_Traffic']):
    print("Creating traffic ratio features...")
    # Avoid division by zero
    df['Arrivals_Ratio'] = df['Arrivals'] / (df['Total_Traffic'] + 1)
    df['Departures_Ratio'] = df['Departures'] / (df['Total_Traffic'] + 1)
    print("- Created Arrivals_Ratio and Departures_Ratio")

# 2. Create month-based seasonal indicators
if 'Month' in df.columns:
    print("Creating seasonal indicators...")
    # Create season indicators (Northern hemisphere)
    # Winter: Dec-Feb (12, 1, 2), Spring: Mar-May (3, 4, 5), 
    # Summer: Jun-Aug (6, 7, 8), Fall: Sep-Nov (9, 10, 11)
    df['Winter'] = df['Month'].isin([12, 1, 2]).astype(int)
    df['Spring'] = df['Month'].isin([3, 4, 5]).astype(int)
    df['Summer'] = df['Month'].isin([6, 7, 8]).astype(int)
    df['Fall'] = df['Month'].isin([9, 10, 11]).astype(int)
    print("- Created seasonal indicators (Winter, Spring, Summer, Fall)")

# 3. Create holiday-traffic interaction features
if 'Total Holidays' in df.columns and 'Total_Traffic' in df.columns:
    print("Creating holiday interaction features...")
    df['Holiday_Traffic_Ratio'] = df['Total Holidays'] / (df['Total_Traffic'] + 1)
    df['Holiday_Present'] = (df['Total Holidays'] > 0).astype(int)
    print("- Created Holiday_Traffic_Ratio and Holiday_Present")

# 4. Calculate logarithmic transformations of key metrics
# This can help with skewed distributions
for col in ['Arrivals', 'Departures', 'Total_Traffic']:
    if col in df.columns:
        df[f'Log_{col}'] = np.log1p(df[col])  # log1p handles zeros
        print(f"- Created Log_{col}")

# 5. Create month and inflation interaction
if all(col in df.columns for col in ['Month', 'Inflation']):
    print("Creating month-inflation interactions...")
    for month in range(1, 13):
        df[f'Month{month}_Inflation'] = ((df['Month'] == month) * df['Inflation']).astype(float)
    print("- Created month-specific inflation features")

# 6. Month-Holiday interaction
if all(col in df.columns for col in ['Month', 'Total Holidays']):
    print("Creating month-holiday interactions...")
    for month in range(1, 13):
        df[f'Month{month}_Holidays'] = ((df['Month'] == month) * df['Total Holidays']).astype(float)
    print("- Created month-specific holiday features")

# Add all new features to our feature list
new_features = [col for col in df.columns if col not in base_feature_columns 
                and col not in categorical_columns_to_drop
                and col not in drop_columns
                and col != 'Traffic_Class']

# Combine base and new features
all_features = base_feature_columns + new_features

# Print new feature summary
print(f"\nCreated {len(new_features)} new engineered features:")
print(", ".join(new_features))

# Define X and y, dropping original categorical columns
X = df[all_features].copy()
y = df['Traffic_Class'].copy()

# Print final feature information
print(f"\nTotal features for modeling: {len(all_features)}")
print(f"- {len(base_feature_columns)} base features")
print(f"- {len(new_features)} engineered features")

# Split data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

# Check for any remaining missing values in features
missing_in_features = X.isnull().sum()
if missing_in_features.sum() > 0:
    print("\nMissing values in feature matrix:")
    print(missing_in_features[missing_in_features > 0])

In [ ]:
# =============================================================================
# MODEL PIPELINE SETUP
# =============================================================================
print("\n=== MODEL PIPELINE SETUP ===")

# Create a preprocessing pipeline that handles missing values and scaling
preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Enhanced features pipeline with polynomial features
poly_preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
])


In [ ]:
# =============================================================================
# HYPERPARAMETER TUNING
# =============================================================================
print("\n=== HYPERPARAMETER TUNING ===")

# Create pipeline with preprocessing and classifier
pipe = Pipeline([
    ('preprocessor', preprocessing),
    ('classifier', LogisticRegression(max_iter=2000, random_state=42))
])

# Parameter grid for tuning
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__solver': ['lbfgs', 'newton-cg'],
    'classifier__class_weight': [None, 'balanced']
}

# Stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform grid search
grid_search = GridSearchCV(
    pipe, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1
)

print("Performing grid search for optimal hyperparameters...")
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

# Store best parameters
best_params = grid_search.best_params_
best_C = best_params['classifier__C']
best_solver = best_params['classifier__solver']
best_class_weight = best_params['classifier__class_weight']

print(f"\nOptimal parameters: C={best_C}, solver={best_solver}, class_weight={best_class_weight}")

In [ ]:
# =============================================================================
# MODEL COMPARISON
# =============================================================================
print("\n=== MODEL COMPARISON ===")

# Define the three models to compare
models = {
    "Base Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(max_iter=2000, random_state=42, solver='lbfgs'))
    ]),
    "Tuned Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ]),
    "Logistic with Enhanced Features": Pipeline([
        ('preprocessor', poly_preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ])
}

# Function to evaluate each model
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Classification Model"):
    """
    Trains and evaluates a classification model for traffic class prediction.
    
    Parameters:
    -----------
    model : The classification model pipeline to evaluate
    X_train, X_test : Training and test feature matrices
    y_train, y_test : Training and test target values
    model_name : Name of the model for reporting purposes
    
    Returns:
    --------
    accuracy : Overall accuracy of the model
    y_pred : Predicted classes for the test set
    trained_model : The trained model object
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Print summary
    print(f"\n===== {model_name} =====")
    print(f"Accuracy: {accuracy:.4f}")
    
    # Generate detailed classification metrics
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Traffic Class')
    plt.xlabel('Predicted Traffic Class')
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_pred, model

# Train and evaluate each model
results = {}
models_trained = {}

# Loop through each model and evaluate
for name, model in models.items():
    # Evaluate the model using the same X_train and X_test for all models
    accuracy, y_pred, trained_model = evaluate_model(
        model, X_train, X_test, y_train, y_test, name
    )
    results[name] = accuracy
    models_trained[name] = trained_model

# Print comparison of model accuracies
print("\n=== FINAL RESULTS ===")
for name, accuracy in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{name}: {accuracy:.4f}")


In [ ]:
# =============================================================================
# BEST MODEL ANALYSIS
# =============================================================================
print("\n=== BEST MODEL ANALYSIS ===")

# Identify the model with the highest accuracy
best_model_name = max(results, key=results.get)
print(f"Best model: {best_model_name} (Accuracy: {results[best_model_name]:.4f})")

# Get the best trained model
best_model = models_trained[best_model_name]

# Get predictions
y_pred_best = best_model.predict(X_test)

# Calculate accuracy per class
class_accuracy = {}
for class_id in np.unique(y_test):
    class_mask = (y_test == class_id)
    class_correct = (y_pred_best[class_mask] == class_id).sum()
    class_total = class_mask.sum()
    class_accuracy[class_id] = class_correct / class_total

print("\nAccuracy by Traffic Class:")
for class_id, acc in sorted(class_accuracy.items()):
    print(f"Class {class_id}: {acc:.4f}")

In [ ]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================
print("\n=== FEATURE IMPORTANCE ANALYSIS ===")

# For logistic regression, we can examine coefficients
if "Logistic" in best_model_name:
    # Get the classifier from the pipeline
    classifier = best_model.named_steps['classifier']
    
    # Get feature names after preprocessing
    if best_model_name == "Logistic with Enhanced Features":
        # For polynomial features, we need to get the transformed feature names
        poly = best_model.named_steps['preprocessor'].named_steps['poly']
        feature_names = poly.get_feature_names_out(input_features=all_features)
    else:
        feature_names = all_features
    
    # Get coefficients
    coefficients = classifier.coef_
    
    # For multiclass, average the absolute coefficient values across classes
    importance = np.abs(coefficients).mean(axis=0)
    
    # Create a DataFrame for visualization
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    # Display top 20 features
    print("\nTop 20 most important features:")
    for i, (feature, importance) in enumerate(zip(feature_importance['Feature'][:20], 
                                                feature_importance['Importance'][:20])):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 10))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title('Top 20 Most Important Features')
    plt.tight_layout()
    plt.show()


In [ ]:
# =============================================================================
# SAVE THE BEST MODEL
# =============================================================================
# Create directory if it doesn't exist
model_dir = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the best model
best_model_file = os.path.join(model_dir, 'traffic_class_predictor_onehot.pkl')
with open(best_model_file, 'wb') as f:
    pickle.dump({
        'model': best_model,
        'features': all_features,
        'encoder': encoder if 'encoder' in locals() else None
    }, f)

print(f"\nBest model saved to {best_model_file}")

In [ ]:
# =============================================================================
# INSIGHTS AND RECOMMENDATIONS
# =============================================================================
# Summarize key findings and provide actionable recommendations

print("\n=== TRAFFIC CLASS PREDICTION INSIGHTS ===")
print("The model predicts traffic volume classes for Singapore air travel with improved")
print("accuracy through advanced feature engineering and restored traffic metrics.")
print("\nKey findings:")
print("1. Restoring key traffic features (Total_Traffic, Arrivals, Departures) and")
print("   creating advanced feature combinations significantly improved model performance.")
print("2. Seasonal patterns, holiday interactions, and traffic ratios provide valuable")
print("   predictive information for identifying different traffic classes.")
print("3. The model achieves substantially better results with this enhanced approach.")
print("\nRecommendations for stakeholders:")
print("1. Airlines should focus on the most predictive features identified in this model")
print("   when optimizing flight schedules.")
print("2. The added seasonal and holiday interaction features can help travel agencies")
print("   better target their promotions during specific time periods.")
print("3. Airport resource allocation can be improved by monitoring these key indicators,")
print("   particularly the traffic ratios and holiday-related features.")

In [ ]:
"""
Singapore Air Traffic Class Prediction Model with One-Hot Encoding

This code builds and evaluates a machine learning model to predict traffic classes based on
travel data to and from Singapore. It implements advanced feature engineering with one-hot
encoding for categorical variables instead of label encoding.

Author: Myat Ko
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import shap

# =============================================================================
# DATA LOADING AND EXPLORATION
# =============================================================================
# First, we load our dataset and examine its basic properties

# Attempt to load the dataset from different possible locations
try:
    df = pd.read_csv("train_data_with_traffic_class.csv")
except:
    # Try alternative paths if the file isn't found
    try:
        df = pd.read_csv("C:/Users/Acer/Dropbox/PC (2)/Documents/Github/traveltrends/Classification/train_data_with_traffic_class.csv")
    except:
        # To adjust this path based on current file structure
        print("Please check the file path and try again.")

# Print dataset summary information
print(f"Dataset shape: {df.shape}")
print(f"Number of traffic classes: {df['Traffic_Class'].nunique()}")
print(f"Traffic classes: {sorted(df['Traffic_Class'].unique())}")

# Check for missing values
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print("\nMissing values in dataset:")
    print(missing_values[missing_values > 0])

# Examine the distribution of traffic classes
traffic_class_counts = df['Traffic_Class'].value_counts().sort_index()
print("\nTraffic class distribution:")
for class_id, count in traffic_class_counts.items():
    percentage = 100 * count / len(df)
    print(f"Class {class_id}: {count} samples ({percentage:.1f}%)")

# =============================================================================
# DATA PREPROCESSING AND FEATURE ENGINEERING
# =============================================================================

print("\n=== DATA PREPROCESSING AND FEATURE ENGINEERING ===")

# First, remove any columns with all NaN values
df = df.dropna(axis=1, how='all')

# Drop Unnecessary features since we are finding traffic_class(our target)
drop_columns = ["Total_Traffic", "Departures", "Arrivals", "Traffic_Class", "Month"]

# Identify categorical columns (object type) and numeric columns
categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and col not in drop_columns]
numeric_columns = [col for col in df.columns if df[col].dtype != 'object' and col not in drop_columns]

print(f"Categorical columns: {categorical_columns}")
print(f"Numeric columns: {numeric_columns}")

# =============================================================================
# KEY CHANGE: Use One-Hot Encoding instead of Label Encoding for categorical variables
# =============================================================================

# Handle categorical features with one-hot encoding
# This prevents introducing arbitrary ordinal relationships
if categorical_columns:
    print("\nApplying one-hot encoding to categorical columns...")
    
    # First, fill missing values in categorical columns
    for col in categorical_columns:
        df[col] = df[col].fillna('Unknown')
    
    # Apply one-hot encoding
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_cats = encoder.fit_transform(df[categorical_columns])
    
    # Get the feature names from the encoder
    encoded_feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded values
    encoded_df = pd.DataFrame(encoded_cats, columns=encoded_feature_names, index=df.index)
    
    # Concatenate with the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)
    
    print(f"- Created {len(encoded_feature_names)} one-hot encoded features")
    
    # Track the encoded columns for later use
    categorical_encoded = list(encoded_feature_names)
    # Keep track of original columns to exclude from model features
    categorical_columns_to_drop = categorical_columns.copy()
else:
    categorical_encoded = []
    categorical_columns_to_drop = []

# Define our base feature columns - both numeric and encoded categorical
base_feature_columns = numeric_columns + categorical_encoded

# =============================================================================
# CREATE ADDITIONAL ADVANCED FEATURES
# =============================================================================
# print("\n=== CREATING ADVANCED FEATURE COMBINATIONS ===")

# Add all new features to our feature list
new_features = [col for col in df.columns if col not in base_feature_columns 
                and col not in categorical_columns_to_drop
                and col not in drop_columns
                and col != 'Traffic_Class']

# Combine base and new features
all_features = base_feature_columns + new_features

# Print new feature summary
print(f"\nCreated {len(new_features)} new engineered features:")
print(", ".join(new_features))

# Define X and y, dropping original categorical columns
X = df[all_features].copy()
y = df['Traffic_Class'].copy()

# Print final feature information
print(f"\nTotal features for modeling: {len(all_features)}")
print(f"- {len(base_feature_columns)} base features")
print(f"- {len(new_features)} engineered features")

#df.to_csv('train_data_with_traffic_class_merged.csv', index=False)

# =============================================================================
# TRAIN-VALIDATION SPLIT
# =============================================================================
print("\n=== TRAIN-VALIDATION SPLIT ===")

# Split into train (70%) and validation (30%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")

# Check for any remaining missing values in features
missing_in_features = X.isnull().sum()
if missing_in_features.sum() > 0:
    print("\nMissing values in feature matrix:")
    print(missing_in_features[missing_in_features > 0])

# =============================================================================
# MODEL PIPELINE SETUP
# =============================================================================
print("\n=== MODEL PIPELINE SETUP ===")

# Create a preprocessing pipeline that handles missing values and scaling
preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Enhanced features pipeline with polynomial features
poly_preprocessing = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
])

# =============================================================================
# HYPERPARAMETER TUNING
# =============================================================================
print("\n=== HYPERPARAMETER TUNING ===")

# Create pipeline with preprocessing and classifier
pipe = Pipeline([
    ('preprocessor', preprocessing),
    ('classifier', LogisticRegression(max_iter=2000, random_state=42))
])

# Parameter grid for tuning
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__solver': ['lbfgs', 'newton-cg'],
    'classifier__class_weight': [None, 'balanced']
}

# Stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform grid search on the training data
grid_search = GridSearchCV(
    pipe, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1
)

print("Performing grid search for optimal hyperparameters...")
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

# Store best parameters
best_params = grid_search.best_params_
best_C = best_params['classifier__C']
best_solver = best_params['classifier__solver']
best_class_weight = best_params['classifier__class_weight']

print(f"\nOptimal parameters: C={best_C}, solver={best_solver}, class_weight={best_class_weight}")

# =============================================================================
# MODEL COMPARISON WITH VALIDATION SET
# =============================================================================
print("\n=== MODEL COMPARISON WITH VALIDATION SET ===")

# Define the three models to compare
models = {
    "Base Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(max_iter=2000, random_state=42, solver='lbfgs'))
    ]),
    "Tuned Logistic Regression": Pipeline([
        ('preprocessor', preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ]),
    "Logistic with Enhanced Features": Pipeline([
        ('preprocessor', poly_preprocessing),
        ('classifier', LogisticRegression(
            C=best_C, solver=best_solver, class_weight=best_class_weight,
            max_iter=2000, random_state=42))
    ])
}

# Function to evaluate each model
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name="Classification Model"):
    """
    Trains and evaluates a classification model using training and validation sets.
    
    Parameters:
    -----------
    model : The classification model pipeline to evaluate
    X_train, X_val : Training and validation feature matrices
    y_train, y_val : Training and validation target values
    model_name : Name of the model for reporting purposes
    
    Returns:
    --------
    metrics : Dictionary containing train and validation accuracy
    y_pred_val : Predicted classes for the validation set
    trained_model : The trained model object
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on train and validation sets
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    
    # Print summary
    print(f"\n===== {model_name} =====")
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Check for overfitting
    train_val_diff = train_accuracy - val_accuracy
    print(f"Train-Validation Accuracy Gap: {train_val_diff:.4f}")
    
    if train_val_diff > 0.05:
        print("WARNING: Possible overfitting detected (Train-Val gap > 5%)")
    
    # Generate detailed classification metrics on validation set
    print("\nValidation Classification Report:")
    print(classification_report(y_val, y_pred_val))
    
    # Create confusion matrix for validation set
    cm_val = confusion_matrix(y_val, y_pred_val)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Validation Confusion Matrix - {model_name}')
    plt.ylabel('True Traffic Class')
    plt.xlabel('Predicted Traffic Class')
    plt.tight_layout()
    plt.show()
    
    # Return metrics and model
    metrics = {
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'train_val_diff': train_val_diff
    }
    
    return metrics, y_pred_val, model

# Train and evaluate each model
results = {}
models_trained = {}

# Loop through each model and evaluate
for name, model in models.items():
    # Evaluate the model using train and validation sets
    metrics, y_pred_val, trained_model = evaluate_model(
        model, X_train, X_val, y_train, y_val, name
    )
    results[name] = metrics
    models_trained[name] = trained_model

# Print comparison of model accuracies
print("\n=== FINAL RESULTS ===")
print("Model                      Train Acc  Val Acc   Train-Val Gap")
print("-------------------------  ---------  --------  -------------")
for name, metrics in sorted(results.items(), key=lambda x: x[1]['val_accuracy'], reverse=True):
    print(f"{name[:25]:<25}  {metrics['train_accuracy']:.4f}    {metrics['val_accuracy']:.4f}    {metrics['train_val_diff']:.4f}")

# =============================================================================
# BEST MODEL ANALYSIS
# =============================================================================
print("\n=== BEST MODEL ANALYSIS ===")

# Get the best model based on validation accuracy
best_model_name = sorted(results.items(), key=lambda x: x[1]['val_accuracy'], reverse=True)[0][0]
print(f"Best model: {best_model_name}")
print(f"Validation Accuracy: {results[best_model_name]['val_accuracy']:.4f}")

# Get the best trained model
best_model = models_trained[best_model_name]

# Get validation predictions
y_pred_val = best_model.predict(X_val)

# Calculate accuracy per class
class_accuracy = {}
for class_id in np.unique(y_val):
    class_mask = (y_val == class_id)
    class_correct = (y_pred_val[class_mask] == class_id).sum()
    class_total = class_mask.sum()
    class_accuracy[class_id] = class_correct / class_total

print("\nAccuracy by Traffic Class:")
for class_id, acc in sorted(class_accuracy.items()):
    print(f"Class {class_id}: {acc:.4f}")

# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================
print("\n=== FEATURE IMPORTANCE ANALYSIS ===")

# For logistic regression, we can examine coefficients
if "Logistic" in best_model_name:
    # Get the classifier from the pipeline
    classifier = best_model.named_steps['classifier']
    
    # Get feature names after preprocessing
    if best_model_name == "Logistic with Enhanced Features":
        # For polynomial features, we need to get the transformed feature names
        poly = best_model.named_steps['preprocessor'].named_steps['poly']
        feature_names = poly.get_feature_names_out(input_features=all_features)
    else:
        feature_names = all_features
    
    # Get coefficients
    coefficients = classifier.coef_
    
    # For multiclass, average the absolute coefficient values across classes
    importance = np.abs(coefficients).mean(axis=0)
    
    # Create a DataFrame for visualization
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    # Display top 20 features
    print("\nTop 20 most important features:")
    for i, (feature, importance) in enumerate(zip(feature_importance['Feature'][:20], 
                                                feature_importance['Importance'][:20])):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 10))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title('Top 20 Most Important Features')
    plt.tight_layout()
    plt.show()




In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

# =============================================================================
# TEST DATA LOADING
# =============================================================================
print("\n=== LOADING TEST DATA ===")

# Attempt to load the test dataset
try:
    test_df = pd.read_csv("test_data.csv")
    print(f"Test dataset shape: {test_df.shape}")
except FileNotFoundError:
    # Try alternative paths if the file isn't found
    try:
        test_df = pd.read_csv("C:/Users/Acer/Dropbox/PC (2)/Documents/Github/traveltrends/Classification/test_data_with_traffic_class.csv")
        print(f"Test dataset shape: {test_df.shape}")
    except FileNotFoundError:
        print("Test data file not found. Please check the file path and try again.")
        exit()

# Check if the test dataset has the target variable
has_target = 'Traffic_Class' in test_df.columns
if has_target:
    print("Test dataset contains 'Traffic_Class' column. Will evaluate model performance.")
    X_test = test_df.drop('Traffic_Class', axis=1)
    y_test = test_df['Traffic_Class']
else:
    print("Test dataset does not contain 'Traffic_Class' column. Will only generate predictions.")
    X_test = test_df.copy()
    y_test = None

# =============================================================================
# LOAD TRAINED MODELS
# =============================================================================
print("\n=== LOADING TRAINED MODELS ===")

# Get the trained models from the previous execution
# These should be the models defined in the 'models_trained' dictionary
models_trained = {
    "Base Logistic Regression": models_trained["Base Logistic Regression"], 
    "Tuned Logistic Regression": models_trained["Tuned Logistic Regression"],
    "Logistic with Enhanced Features": models_trained["Logistic with Enhanced Features"]
}

# =============================================================================
# PREDICTION FUNCTION
# =============================================================================
def predict_and_evaluate(model, X_test, y_test=None, model_name="Classification Model"):
    """
    Makes predictions using the trained model and evaluates if target is available.
    
    Parameters:
    -----------
    model : The trained classification model
    X_test : Test feature matrix
    y_test : Test target values (optional)
    model_name : Name of the model for reporting purposes
    
    Returns:
    --------
    y_pred : Predicted classes for the test set
    """
    print(f"\n===== {model_name} =====")
    
    # Make predictions
    try:
        y_pred = model.predict(X_test)
        print(f"Successfully generated predictions for {len(y_pred)} samples")
        
        # If we have target values, evaluate the model
        if y_test is not None:
            test_accuracy = accuracy_score(y_test, y_pred)
            print(f"Test Accuracy: {test_accuracy:.4f}")
            
            # Generate detailed classification metrics
            print("\nTest Classification Report:")
            print(classification_report(y_test, y_pred))
            
            # Create confusion matrix
            cm_test = confusion_matrix(y_test, y_pred)
            
            # Plot confusion matrix
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
            plt.title(f'Test Confusion Matrix - {model_name}')
            plt.ylabel('True Traffic Class')
            plt.xlabel('Predicted Traffic Class')
            plt.tight_layout()
            plt.savefig(f'{model_name.replace(" ", "_")}_confusion_matrix.png')
            plt.show()
            
            # Calculate accuracy per class
            class_accuracy = {}
            for class_id in np.unique(y_test):
                class_mask = (y_test == class_id)
                class_correct = (y_pred[class_mask] == class_id).sum()
                class_total = class_mask.sum()
                class_accuracy[class_id] = class_correct / class_total
            
            print("\nAccuracy by Traffic Class:")
            for class_id, acc in sorted(class_accuracy.items()):
                print(f"Class {class_id}: {acc:.4f}")
        
        return y_pred
    
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        return None

# =============================================================================
# MAKE PREDICTIONS WITH ALL MODELS
# =============================================================================
print("\n=== MAKING PREDICTIONS WITH ALL MODELS ===")

results = {}

# Test each model
for name, model in models_trained.items():
    try:
        y_pred = predict_and_evaluate(model, X_test, y_test, name)
        results[name] = y_pred
        
        # Save predictions to CSV
        if y_pred is not None:
            predictions_df = pd.DataFrame({
                'Predicted_Traffic_Class': y_pred
            })
            
            # If we have original data, include it in the output
            if has_target:
                for col in test_df.columns:
                    if col != 'Traffic_Class':
                        predictions_df[col] = test_df[col]
                predictions_df['True_Traffic_Class'] = y_test
            else:
                for col in test_df.columns:
                    predictions_df[col] = test_df[col]
            
            # Save to CSV
            output_filename = f'{name.replace(" ", "_")}_predictions.csv'
            predictions_df.to_csv(output_filename, index=False)
            print(f"Predictions saved to {output_filename}")
    
    except Exception as e:
        print(f"Error processing model {name}: {str(e)}")

# =============================================================================
# COMPARE MODEL PERFORMANCE
# =============================================================================
if has_target:
    print("\n=== TEST PERFORMANCE COMPARISON ===")
    print("Model                      Test Accuracy")
    print("-------------------------  -------------")
    
    model_accuracies = {}
    for name, y_pred in results.items():
        if y_pred is not None:
            accuracy = accuracy_score(y_test, y_pred)
            model_accuracies[name] = accuracy
    
    # Sort by accuracy
    for name, accuracy in sorted(model_accuracies.items(), key=lambda x: x[1], reverse=True):
        print(f"{name[:25]:<25}  {accuracy:.4f}")
    
    # Find the best model
    best_model_name = max(model_accuracies.items(), key=lambda x: x[1])[0]
    print(f"\nBest model on test data: {best_model_name}")
    print(f"Test Accuracy: {model_accuracies[best_model_name]:.4f}")


print("\n=== TESTING COMPLETE ===")


=== LOADING TEST DATA ===
Test dataset shape: (449, 11)
Test dataset contains 'Traffic_Class' column. Will evaluate model performance.

=== LOADING TRAINED MODELS & ENCODERS ===
Loaded OneHotEncoder from file.
Processed test data shape: (449, 23)
Features: ['Year', 'Total Holidays', 'Inflation', 'Month_sin', 'Month_cos', 'Country_Europe', 'Country_France', 'Country_Germany', 'Country_Hong Kong', 'Country_Indonesia', 'Country_Japan', 'Country_Mainland China', 'Country_Malaysia', 'Country_Middle East', 'Country_North America', 'Country_North East Asia', 'Country_Oceania', 'Country_Philippines', 'Country_South Asia', 'Country_South East Asia', 'Country_Thailand', 'Country_United Kingdom', 'Country_Vietnam']

=== LOADING TRAINED MODELS ===
One or more model files not found. Please ensure models have been trained and saved.

=== MAKING PREDICTIONS WITH ALL MODELS ===


NameError: name 'models_trained' is not defined

: 